In [146]:
import pandas as pd

In [147]:
csv_1 = pd.read_csv('if_else(1).csv', index_col=0)
csv_2 = pd.read_csv('if_else(2).csv', index_col=0)

In [148]:
mask = csv_1.index.isin(csv_2.index)
csv_1.loc[mask, 'rationales'] = csv_2.loc[csv_2.index.isin(csv_1.index), 'rationales']

In [149]:
csv_1

,premise,hypothesis,rationales
0,A man with a white camp is standing on a platf...,A man is hanging off of a clothesline.,1. Premise: A man with a white camp is standin...
1,Two girls wearing skirts like the flag of Grea...,a couple want to eat a flag.,2. Premise: Two girls wearing skirts like the ...
2,Dog leaps from water while woman kneels on the...,A woman and dog sit on a dock together.,3. Premise: Dog leaps from water while woman k...
3,A bald man in an orange shirt and jeans is vac...,The clothed man is cleaning the carpet,4. Premise: A bald man in an orange shirt and ...
4,A boy stands beside a railing at a go kart track.,The boy is going to ride a go kart.,5. Premise: A boy stands beside a railing at a...
...,...,...,...
5587,A woman is tugging on a white sheet and laughing,The woman is changing the sheets on her bed.,8. Premise: A woman is tugging on a white shee...
5588,A man is looking over a small cliff.,A woman is looking,9. Premise: A man is looking over a small clif...
5589,A man in goggle swimming.,A man swimming in the water with goggles.,10. Premise: A man in goggles swimming.\n H...
5590,Two men are standing in front of musical mixin...,There are people indoors.,"The correct answer is ""entailment."" The premi..."


In [150]:
from datasets import load_dataset
import pandas as pd

dataset = load_dataset("esnli")
train_df = pd.DataFrame(dataset['train']).sample(frac=1/100, random_state=0)
validation = pd.DataFrame(dataset['validation']).sample(frac=1/100, random_state=0)

train_df = pd.concat([train_df, validation], ignore_index=True)

In [151]:
csv_1['label'] = train_df['label']

In [152]:
csv_1['split'] = csv_1.rationales.apply(lambda x: x.lower().split('most likely answer'))

In [153]:
csv_1.split = csv_1.split.apply(lambda x : x[1] if len(x) > 1 else x[0])

In [154]:
csv_1['correct_index'] = [{} for _ in range(len(csv_1))]
for index, row in csv_1.iterrows():
    for choice in ["entailment", "neutral", "contradiction", 'entailed', 'contradicts', 'entails', 'contradicted', 'contradictory']:
        if choice in row['split']:
            row['correct_index'][choice] = row['split'].lower().index(choice)

In [155]:
import numpy as np
csv_1['LLM_answer'] =csv_1.correct_index.apply(lambda x: min(x, key=x.get) if len(x) > 0 else np.nan)

In [156]:
csv_1[csv_1.LLM_answer.isna()]

,premise,hypothesis,rationales,label,split,correct_index,LLM_answer


In [157]:
csv_1[csv_1.LLM_answer == 'entailed'] = 'entailment'
csv_1[csv_1.LLM_answer == 'entails'] = 'entailment'
csv_1[csv_1.LLM_answer == 'contradicts'] = 'contradiction'
csv_1[csv_1.LLM_answer == 'contradicted'] = 'contradiction'
csv_1[csv_1.LLM_answer == 'contradictory'] = 'contradiction'

In [163]:
csv_1.loc[csv_1.label == 0, 'label'] = 'entailment'
csv_1.loc[csv_1.label == 1, 'label'] = 'neutral'
csv_1.loc[csv_1.label == 2, 'label'] = 'contradiction'

In [165]:
sum(csv_1.label == csv_1.LLM_answer) / len(csv_1)

0.6369814020028612

In [177]:
csv_1['question'] = csv_1['premise'] + '</s>' + csv_1['hypothesis']

In [169]:
csv_1.to_csv('if_else - full.csv', index=False)